<a href="https://colab.research.google.com/github/mzk8888/AI/blob/main/qa4u3day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

まずは量子アニーリングマシンを使うための準備とシミュレータとしてOpenJijを利用する準備としてpip installをしておきましょう

In [ ]:
pip install dwave-ocean-sdk

In [ ]:
pip install openjij

今回の講義内容は以下のURLにある情報を参考にしました。
https://www.tensorflow.org/tutorials/audio/music_generation?hl=ja

## 楽曲を扱うためのライブラリ
さらに今回は音楽生成が楽しくなるように楽譜の表示をするためのライブラリとしてmusic21を用意します。

In [ ]:
!pip install --upgrade music21

ただこれではうまくいかないのでちょっとおまじない。
music21の背景で必要となるmusescoreをシステムにインストールします。

In [ ]:
!apt-get install musescore

さらに必要なものとしてxvfbもインストールしましょう。

In [ ]:
!apt-get install xvfb

これらの必要なものをインストールしたら、おまじない。
新しい環境で起動しましょうと言う意味です。

In [ ]:
!sh -e /etc/init.d/x11-common start

In [ ]:
import os

In [ ]:
os.putenv('DISPLAY', ':99.0')

In [ ]:
!start-stop-daemon --start --pidfile /var/run/xvfb.pid --make-pidfile --background --exec /usr/bin/Xvfb -- :99 -screen 0 1024x768x24 -ac +extension GLX +render -noreset

ここからが本番です。
まずはmusic21を使えるように準備しましょう。
from music21 import *で特に宣言なくmusic21にある関数を利用することができるようになります。

In [ ]:
from music21 import *

さらに作業環境のためのおまじない

In [ ]:
us = environment.UserSettings()

In [ ]:
us['musescoreDirectPNGPath'] = '/usr/bin/mscore'

In [ ]:
us['musicxmlPath'] = '/usr/bin/mscore'

In [ ]:
us['directoryScratch'] = '/tmp'

それではまずお試しに楽譜を表示してみましょう。
以下のconverter.parseのなかにある4/4は4/4拍子で、c4やd4などは音階（4や8は4分音符、8分音符の意味）です。rは休符。
さてこれは何の曲でしょうか？

1.   リスト項目
2.   リスト項目



In [ ]:
cp = converter.parse('tinyNotation: 4/4 c4 d4 e4 f4 e4 d4 c4 r e4 f4 g4 a4 g4 f4 e4 r c4 r c4 r c4 r c4 r c8 c8 d8 d8 e8 e8 f8 f8 e4 d4 c4 r')

それではこれを５線符に並べて楽譜を表示してみましょう

In [ ]:
cp.show()

楽譜でも良いですし音階の上下だけ記号的に表したピアノロール表示がコンピュータによる楽曲生成にはよく用いられます。

In [ ]:
cp.plot()

音階が何回出てきたのか頻度を調べるヒストグラム表示も可能です。

In [ ]:
cp.plot('histogram', 'pitch')

さらにはこれを音声として実際に再生をすることもできます。
これが先ほどのおまじないの効果です。

In [ ]:
cp.show('midi')

## 楽曲のデータ分析
さてそうした下準備を終えたら、次は世の中の楽曲についてのデータ分析をしてみましょう。
今回は日本ではファミリーコンピュータ（通称：ファミコン）で知られる任天堂の家庭用ゲーム機で発表されたゲームの楽曲を利用します。
同時発音数が3つまでと言う制限がありシンプルなデータであると言うのが選定理由です。世界的にはNES（Nintendo Entertainment System）で知られています。

以下のgithubレポジトリにNESのMIDEデータがありますので、これを利用しましょう。
https://github.com/chrisdonahue/nesmdb
ダウンロードしたのち、個々のGoogle colab経由でアップロードして利用しましょう。
利用するのはSeparated Score Formatです。

In [ ]:
from google.colab import files
uploaded = files.upload()

（圧縮されたファイルをそのまま）アップロードをしたらそのまま解凍をします。

In [ ]:
!tar -zxvf nesmdb24_seprsco.tar.gz

これでデータ分析をする準備ができました。
いつも利用するnumpyはもちろん、ファイル名関係を扱うライブラリのglobを利用します。

In [ ]:
import numpy as np
import glob
files = glob.glob(r"/content/nesmdb24_seprsco/train/*")

ファイル名のリストをfilesに準備しておきました。
これをpickleというライブラリを用いて次々と開いてGoogle Colab上で利用できるようにしましょう。たまに空ファイルがあるので、seprsco.size > 0のものだけを読み込むという例外処理を行なっています。

In [ ]:
length = 8

今回は楽曲の全てを学ぶとかではなく、フレーズ的に一部分を生成することができることを目指します。
length=8として、4/4拍子で8個の音を生成することを目指します。
以下で呼び出すseprsco.shape[0]で曲の長さを調べることができます。

In [ ]:
import pickle
data_dict = {}
for file in files:
  with open(file, 'rb') as f:
    rate, nsamps, seprsco = pickle.load(f)
    if seprsco.shape[0] > 0 and seprsco.shape[0] > length:
      data_dict[file] = [rate, nsamps, seprsco]
      print(file, nsamps, seprsco.shape)

ファイル名をキーにしたdata_dict（dict形式）に格納することにしました。


In [ ]:
key_list = list(data_dict.keys())

ちょっと試しに一曲だけピアノロールを見てみましょう。

In [ ]:
piano_rolls = data_dict[key_list[0]][2]

In [ ]:
piano_rolls

数十の数値がたくさん並んでいるかと思います。
まず列が楽器とも言いましょうか。ファミコンの音源1,2,3,4のそれぞれの音に対応します。
パルス波2つと三角波、パーカッション部分です。

two pulse-wave generators (P1, P2), a triangle-wave generator (TR), a percussive noise generator (NO)

この数値は音階の値で12毎にオクターブがあがります。
12は半音を含めたCDEFGAB（ドレミファソラシ）までの音です。
つまりピアノの鍵盤の数です。
オクターブは忘れてこの数値がどの鍵盤を押したら出るのかを調べるのはnp.modなどを使うと良いですね。

まず曲の全体の長さを調べて、
それを超えない範囲で適当にlength分の音情報を抜き取る練習をしてみます。
np.random.randintを利用すれば良いですね。

In [ ]:
music_length = len(piano_rolls[:,0])
mstart = np.random.randint(0,music_length-length)
piano_roll = np.mod(piano_rolls[mstart:mstart+length,0],12)
print(piano_roll)

np.modを利用して12音のフレーズを抜き出しました。

これでいくらでも整数値で0-11までのシーケンスを出すことができますね。
音楽ですから特に前後の関係が大事であるわけで、
何かの音が鳴ったら、その音の次にどんな音が来やすいのか、前の音がこうだったら次はどんな音になりやすいのか。その相互作用の関係が大事なはずです。
そこを学ぶボルツマンマシンを実行しましょう。

次にこれを縦に13個・横に8個並ぶ0と1の行列にしていきましょう。12音+休符で合計13個が縦に並びます。
指定された数字の場所に1を立てれば良いですね。


休符かそれ以外かで役割が違うのでその処理を行う関数を用意します。

In [ ]:
def create_roll(piano_roll):
  ans = np.zeros(piano_roll.shape[0])
  itemp0 = np.where(piano_roll==0)
  itemp1 = np.where(piano_roll!=0)
  ans[itemp0] = 0 #0で休符
  ans[itemp1] = np.mod(piano_roll[itemp1]-1,12) + 1 #1-12で音階
  return ans

In [ ]:
binary_mat = np.zeros([13,length])
for t in range(length):
  temp_roll = create_roll(piano_roll)
  binary_mat[int(temp_roll[t]),t] = 1
print(binary_mat)

こうすればQUBOから生成された0と1の結果と見ることができます。
データは実際こうなっていたから、量子アニーリングマシンにも似たような結果を出せるように調整をする。
それがボルツマンマシンでやることでしたね。

これを楽器毎、たくさんのデータを用意すれば良いですね。
また1つ1つのデータをベクトルに直す（0と1が１列に並んだものにする）必要があります。
そのためにこの0と1の行列をベクトルに直しましょう。
そのためにflattenというのを利用すると良いです。

In [ ]:
binary_vec = binary_mat.flatten()
print(binary_vec)

元に戻すためにはreshapeをします。
ここは画像の場合と同じですね。

In [ ]:
print(binary_vec.reshape(13,8))

それでは準備完了です。
大量にある楽曲データを全て0と1のベクトルにして集めましょう。


In [ ]:
num_data = 100000
binary_matrix = np.zeros((4,num_data, length*13))
random_nums = np.random.randint(0,len(key_list),num_data)
for k in range(num_data):
  piano_rolls = data_dict[key_list[random_nums[k]]][2]
  music_length = len(piano_rolls[:,0])
  mstart = np.random.randint(0,music_length-length)
  for m in range(4):
    binary_mat = np.zeros([13,length])
    piano_roll = piano_rolls[mstart:mstart+length,m]
    temp_roll = create_roll(piano_roll)
    for t in range(length):
      binary_mat[int(temp_roll[t]),t] = 1
    binary_vec = binary_mat.flatten()
    binary_matrix[m,k,:] = binary_vec

これでデータの準備が整いました。
それではボルツマンマシンを実行することにしましょう。
まずは量子アニーリングマシンのセッティングをしてしまいましょう。

In [ ]:
from dwave.system import DWaveSampler,FixedEmbeddingComposite
token = "XXXX"
dw_sampler = DWaveSampler(solver='Advantage_system4.1', token=token)

まずは埋め込み（Embedding）を実行しておきましょう。
適切なサイズのQUBOを作成しておきます。

In [ ]:
N = 13*length

In [ ]:
from minorminer import find_embedding

adj = {}
for i in range(N):
  for j in range(N):
    adj[(i,j)] = 1

embedding = find_embedding(adj, dw_sampler.edgelist)

出来上がった埋め込みをFixedしてサンプラーを作りましょう。

In [ ]:
sampler = FixedEmbeddingComposite(dw_sampler, embedding)

これで量子アニーリングマシンの準備もできました。
それでは前回のボルツマンマシンの要領で学習するとしましょう。
今回は楽器の種類別に実行する必要があります。
そこでm=0として楽器をまずは1番目のものを指定しておきます。

In [ ]:
m = 0
qubo_data = np.zeros((N,N))
for k in range(num_data):
  qubo_data = qubo_data + np.outer(binary_matrix[m,k,:],binary_matrix[m,k,:])/num_data

In [ ]:
num_reads = 100
def comp_model(sampleset,num_reads=num_reads):
  qubo_model = np.zeros((N,N))
  for k in range(num_reads):
    qubo_model = qubo_model + np.outer(sampleset.record[k][0],sampleset.record[k][0])/ num_reads
  return qubo_model

これで準備完了です。
それでは前回のボルツマンマシンのときと同様に学習の設定をしておきましょう。

In [ ]:
Tall = 50
eta = 0.1
qubo = np.zeros([N,N])

In [ ]:
for t in range(Tall):
  sampleset = sampler.sample_qubo(qubo, num_reads=num_reads, answer_mode = "raw")
  #sampleset = sampler.sample_qubo(qubo, num_reads=num_reads) シミュレータの場合
  qubo_model = comp_model(sampleset)
  qubo = qubo - eta*(qubo_data - qubo_model)

出来上がったquboに基づいて楽曲を試しに生成してみましょう。
生成するのは量子アニーリングマシンにできあがったquboを入力すれば良いですね。

現時点でどんなものが生成されているのかを見るには、sampleset.recordを実際に見てみると良いでしょう。

In [ ]:
sampleset.record[0][0].reshape(13,length)

たまに精度が悪く2音以上出力されていることもあります。
そういうときは縦方向に足したものは1個までという制限を設けて出力をさせます。
量子アニーリングを利用する場合には罰金法という方法でそれを誘導します。

In [ ]:
penalty = np.zeros([N,N])

for t in range(length):
  for i in range(13):
    for j in range(13):
      penalty[i*length + t,j*length + t] += 1
      if i == j:
        penalty[i*length + t,j*length + t] -= 2

In [ ]:
from openjij import SASampler
samplerSA = SASampler()

In [ ]:
lam = 0.5
sampleset = sampler.sample_qubo(qubo + lam*penalty, num_reads=num_reads)

In [ ]:
sampleset.record[1][0].reshape(13,length)

In [ ]:
note_list = []
temp_mat = sampleset.record[0][0].reshape(13,length)
for t in range(length):
  note_list.append(np.argmax(temp_mat[:,t]))

In [ ]:
note_list

うまく幾つかの結果がでてきたらあとは実際に音を鳴らすというところである。
そこでmusic21を再び利用しよう。
そのために基本は四分音符であるとして0-11の音をCDEF...と割り当てていくことにする。

In [ ]:
note_dict = {}
note_dict[0] = 'r'
note_dict[1] = 'c'
note_dict[2] = 'c#'
note_dict[3] = 'd'
note_dict[4] = 'd#'
note_dict[5] = 'e'
note_dict[6] = 'f'
note_dict[7] = 'f#'
note_dict[8] = 'g'
note_dict[9] = 'g#'
note_dict[10] = 'a'
note_dict[11] = 'a#'
note_dict[12] = 'b'

まずは楽譜を作ってみましょう。サンプルにあるように

'tinyNotation: 4/4 c4 d4 e4 f4 e4 d4 c4 r e4 f4 g4 a4 g4 f4 e4 r c4 r c4 r c4 r c4 r c8 c8 d8 d8 e8 e8 f8 f8 e4 d4 c4 r'

という形で書けば良いので、'tinyNotation: 4/4'
までは先に入力しておき、その後に文字列を追加していき楽譜としましょう。

In [ ]:
gen_score = 'tinyNotation: 4/4'
for t in range(length):
  gen_score += ' ' + note_dict[note_list[t]]
print(gen_score)

以下のようにconverter.parseで楽譜やMIDIでの再生が可能な形式に変換します。

In [ ]:
cp = converter.parse(gen_score)

まずは楽譜を見てみましょう！

In [ ]:
cp.show()

では実際に演奏をしてみましょう！
いかがでしたでしょうか？

In [ ]:
cp.show('midi')

それではD-Wave Advantageからの出力結果を次々と追記して
一連の楽曲を聴いてみましょう。
note_listではなく直接量子アニーリングマシンからの出力結果をgen_scoreに伝えていきます。

In [ ]:
gen_score = 'tinyNotation: 4/4'
for k in range(num_reads//4):
  temp_mat = sampleset.record[k][0].reshape(13,length)
  for t in range(length):
    gen_score += ' ' + note_dict[np.argmax(temp_mat[:,t])]
print(gen_score)

この結果をconverter.parseしてみましょう。

In [ ]:
cp = converter.parse(gen_score)

世界初。量子アニーリングマシンが奏でるNES musicです！！

In [ ]:
cp.show('midi')

楽譜も表示できますし

In [ ]:
cp.show()

この楽譜をpng等で保存することもできます。

In [ ]:
cp.write('midi',"./QAsong.mid")

こちらのmidiファイルをwavファイルに切り替えることも可能です。

まずはちょっとおまじないから。
fluidsynthというプログラムをインストールします。これでmidi2audioが利用できるようになります。

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install midi2audio

ちょっとしたおまじないの後に以下のように元ファイル名と変換後のファイル名を指定します。

In [ ]:
from midi2audio import FluidSynth
fs = FluidSynth(sound_font='font.sf2')
fs.midi_to_audio('QAsong.mid', 'QAsong.wav')

保存されているwavファイルを再生するにはこちら。バッチリですね！

In [ ]:
from IPython.display import Audio
Audio('QAsong.wav')

## おまけ

ここまでは8音を繰り返して出力しているだけですね。
前の音が何々だったから、次はこうする、という続けていくような出力しなければ音楽らしくなりませんね。

まずは最初の出力は適当に量子アニーリングマシンの出力結果を利用しましょう。

In [ ]:
temp_list = []
temp_mat = sampleset.record[0][0].reshape(13,length)
for t in range(length):
  temp_list.append(temp_mat[:,t])

このうちの最後の4音を利用して次の音を生成するというスキームにしていきましょう。

まずその４音だけを抜き取っておきます。

In [ ]:
last4 = np.array(temp_list[-4:]).flatten()

4音と13（12音階＋休符）だけquboを抜き取ります。

quboから同じように前半4音を取り出し、そこに今抜き取った4音を掛け算します。その上で次の4音にかかるquboを抜き取ります。
そうすると次の4音分はどれが良いのか、傾向としてのバイアスを得ることができます。

In [ ]:
next_vec = np.dot(last4,qubo[:-4*13,-4*13:])

これを後半4音を与えるquboに付け足します。

In [ ]:
next_qubo = qubo + np.diag(np.concatenate([next_vec,np.zeros(4*13)]))

In [ ]:
lam = 0.5
sampleset = sampler.sample_qubo(next_qubo + lam*penalty, num_reads=1)

これでまた8音が生成されました。
最後の4つをまた採用して続けていきましょう。

In [ ]:
for step in range(Tall):
  temp_mat = sampleset.record[0][0].reshape(13,length)
  for t in range(length):
    temp_list.append(temp_mat[:,t])
  last4 = np.array(temp_list[-4:]).flatten()
  next_vec = np.dot(last4,qubo[:-4*13,-4*13:])
  next_qubo = qubo + np.diag(np.concatenate([next_vec,np.zeros(4*13)]))
  sampleset = sampler.sample_qubo(next_qubo + lam*penalty, num_reads=1)

これで出来上がったtemp_listから楽譜を作ってみましょう。

In [ ]:
note_list = []
for t in range(len(temp_list)):
  note_list.append(np.argmax(temp_list[t]))

In [ ]:
gen_score = 'tinyNotation: 4/4'
for t in range(len(note_list)):
  gen_score += ' ' + note_dict[note_list[t]]
print(gen_score)

In [ ]:
cp = converter.parse(gen_score)

In [ ]:
cp.show('midi')

In [ ]:
cp.show()

In [ ]:
cp.write('midi',"./QAsong.mid")